In [1]:
img_path = "C:/Users/KangDongha/Desktop/junior/aug"

In [2]:
import os
import cv2
import numpy as np

In [4]:
count = 0
images = []
classNo = []
File_List = os.listdir(img_path)
print("Total Classes :",len(File_List))
Classes=len(File_List)
for x in range (0,len(File_List)):
    Pic_List = os.listdir(img_path+"/"+str(count)) 
    for y in Pic_List:
        curImg = cv2.imread(img_path+"/"+str(count)+"/"+y)
        images.append(curImg)
        classNo.append(count) 
    count += 1
images = np.array(images)
classNo = np.array(classNo)

Total Classes : 3


In [5]:
from sklearn.model_selection import train_test_split

testRatio = 0.2
validationRatio = 0.2

X_train, X_test, y_train, y_test = train_test_split(images, classNo, test_size=testRatio)
X_train, X_validation, y_train, y_validation = train_test_split(X_train, y_train, test_size=validationRatio)

In [6]:
def preprocessing(img):
    img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    img = cv2.equalizeHist(img)
    img = img/255
    return img

In [7]:
X_train=np.array(list(map(preprocessing,X_train)))
X_validation=np.array(list(map(preprocessing,X_validation)))
X_test=np.array(list(map(preprocessing,X_test)))

In [8]:
X_train=X_train.reshape(X_train.shape[0],X_train.shape[1],X_train.shape[2],1)
X_validation=X_validation.reshape(X_validation.shape[0],X_validation.shape[1],X_validation.shape[2],1)
X_test=X_test.reshape(X_test.shape[0],X_test.shape[1],X_test.shape[2],1)

In [9]:
from keras.preprocessing.image import ImageDataGenerator

dataGen= ImageDataGenerator(width_shift_range=0.1,
                            height_shift_range=0.1,
                            shear_range=0.1,  
                            rotation_range=10)  
dataGen.fit(X_train)

In [10]:
from keras.utils.np_utils import to_categorical

y_train = to_categorical(y_train,Classes)
y_validation = to_categorical(y_validation,Classes)
y_test = to_categorical(y_test,Classes)

In [11]:
from keras.models import Sequential 
from keras.layers import Dense ,Dropout, Flatten 
from keras.optimizers import Adam
from keras.layers.convolutional import Conv2D, MaxPooling2D

image_size = (32,32,3)

In [12]:
def myModel():
    no_Of_Filters=60
    size_of_Filter=(5,5)
    size_of_Filter2=(3,3)
    size_of_pool=(2,2)
    no_Of_Nodes = 500
    model= Sequential()
    model.add((Conv2D(no_Of_Filters,size_of_Filter,
                      input_shape=(image_size[0],image_size[1],1),
                      activation='relu')))
    model.add((Conv2D(no_Of_Filters, size_of_Filter, activation='relu')))
    model.add(MaxPooling2D(pool_size=size_of_pool))
 
    model.add((Conv2D(no_Of_Filters//2, size_of_Filter2, activation='relu')))
    model.add((Conv2D(no_Of_Filters//2, size_of_Filter2, activation='relu')))
    model.add(MaxPooling2D(pool_size=size_of_pool))
    model.add(Dropout(0.5))
 
    model.add(Flatten())
    model.add(Dense(no_Of_Nodes,activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(Classes,activation='softmax'))
    
    model.compile(Adam(learning_rate=0.001),loss='categorical_crossentropy',metrics=['accuracy'])
    return model

In [13]:
model = myModel()

In [14]:
import math

batch_size_val = 20
steps_per_epoch_val = math.floor(len(X_train)/batch_size_val)
epochs_val=50

In [15]:
result=model.fit(dataGen.flow(X_train,y_train,batch_size=batch_size_val),
                 steps_per_epoch=steps_per_epoch_val,epochs=epochs_val,
                 validation_data=(X_validation,y_validation),shuffle=1)

Epoch 1/50
8/8 [==============================] - 1s 58ms/step - loss: 1.1224 - accuracy: 0.3490 - val_loss: 1.0954 - val_accuracy: 0.4651
Epoch 2/50
8/8 [==============================] - 0s 38ms/step - loss: 1.1057 - accuracy: 0.3356 - val_loss: 1.0955 - val_accuracy: 0.3953
Epoch 3/50
8/8 [==============================] - 0s 38ms/step - loss: 1.1038 - accuracy: 0.3020 - val_loss: 1.0979 - val_accuracy: 0.3721
Epoch 4/50
8/8 [==============================] - 0s 41ms/step - loss: 1.0956 - accuracy: 0.3624 - val_loss: 1.0906 - val_accuracy: 0.3256
Epoch 5/50
8/8 [==============================] - 0s 40ms/step - loss: 1.0859 - accuracy: 0.4295 - val_loss: 1.0722 - val_accuracy: 0.4651
Epoch 6/50
8/8 [==============================] - 0s 41ms/step - loss: 1.0433 - accuracy: 0.4899 - val_loss: 0.9977 - val_accuracy: 0.5116
Epoch 7/50
8/8 [==============================] - 0s 40ms/step - loss: 0.9225 - accuracy: 0.5772 - val_loss: 0.8405 - val_accuracy: 0.4884
Epoch 8/50
8/8 [===========

In [16]:
score = model.evaluate(X_test,y_test,verbose=0)
print('Test Score:',score[0])
print('Test Accuracy:',score[1])

model_save_path = 'C:/Users/KangDongha/Desktop/junior/model_trained_data.h5'
model.save(model_save_path)

Test Score: 0.0003304436104372144
Test Accuracy: 1.0
